In [1]:
%load_ext autoreload
%autoreload 2

import pprint
pp = pprint.PrettyPrinter(indent=2)

In [2]:
%run create-blocks.ipynb # data will be in prioritized_blocks
# pp.pprint(prioritized_blocks) # take a look
# pp.pprint(token_to_block_dict) # and another look
# print(token_array, len(token_array)) # and one more look

# (THIS IS NO LONGER TRUE) Desired multi-skiplist

1. Keys are offsets in witnesses (up to one key for each and every witness)
1. Values are variant-graph nodes

Some variant graph nodes do not have values for all witnesses. If witness is not in current node,
can go to different node (on same or different level). 

In [3]:
#
# This file is part of Bluepass. Bluepass is Copyright (c) 2012-2014
# Geert Jansen.
#
# Bluepass is free software available under the GNU General Public License,
# version 3. See the file LICENSE distributed with this file for the exact
# licensing terms.

from __future__ import absolute_import, print_function

import os
import sys
import math
import random

__all__ = ['SkipList']


# The following functions are debugging functions. They are available only when
# Python is not started with -O.

if __debug__:

    def fmtnode(node):
        """Format a single skiplist node."""
        level = max(1, len(node) - 3)
        skip = '(none)' if level == 1 else node[-1]
        return '<Node(level={}, key={}, value={}, skip={})>' \
                    .format(level, node[0], node[1], skip)

    def dump(sl, file=sys.stdout):
        """Dump a skiplist to standard output."""
        print('== Dumping skiplist {0!r}'.format(sl), file=file)
        print('Level: {}/{}'.format(sl.level, sl.maxlevel), file=file)
        print('Size: {}'.format(len(sl)), file=file)
        node = sl._head
        print('{0} (head)'.format(fmtnode(node)), file=file)
        node = node[2]
        avglvl = avglen = avgsiz = 0
        while node is not sl._tail:
            print('{0}'.format(fmtnode(node)), file=file)
            node = node[2]
            avglvl += max(1, len(node) - 3)
            avglen += len(node)
            avgsiz += nodesize(node)
        print('{0} (tail)'.format(fmtnode(node)), file=file)
        print('Avg level: {:.2f}'.format(avglvl/len(sl)), file=file)
        print('Avg node len: {:.2f}'.format(avglen/len(sl)), file=file)
        print('Avg node memory: {:.2f}'.format(avgsiz/len(sl)), file=file)
        print(file=file)

    def check(sl):
        """Check the internal structure of a skiplist."""
        level = sl.maxlevel
        assert level > 0
        while sl._head[1+level] is sl._tail and level > 1:
            level -= 1
        assert level == sl.level
        assert sl._head[0] is sl._head[1] is None
        assert sl._head[-1] == 0
        pos = 0
        node = sl._head
        inbound = {id(sl._head): 0, id(sl._tail): len(sl)}
        while node is not sl._tail:
            assert isinstance(node, list)
            level = min(sl.level, max(1, len(node)-3))
            assert 1 <= level <= sl.maxlevel
            for i in range(1, level):
                fnode = node[2+i]
                flevel = min(sl.level, max(1, len(fnode)-3))
                if i == flevel-1:
                    inbound[id(fnode)] = pos
            if level > 1:
                assert id(node) in inbound
                assert pos == inbound[id(node)] + node[-1]
            for i in range(level):
                fnode = node[2+i]
                assert isinstance(fnode, list)
                level = max(1, len(node) - 3)
                assert level >= i+1
            node = node[2]
            pos += 1
        assert sl._tail[0] is None
        assert sl._tail[1] is None
        for i in range(sl.maxlevel):
            assert sl._tail[2+i] is None
        assert len(sl) == inbound[id(sl._tail)] + node[-1]

    def nodesize(node):
        """Return the size of a skiplist node."""
        size = sys.getsizeof(node)
        size += sys.getsizeof(node[0])
        size += sys.getsizeof(node[1])
        # All elements in [3:-1] are references so don't count
        if len(node) > 3:
            size += sys.getsizeof(node[-1])
        return size

    def getsize(sl):
        """Return total size of a skiplist."""
        size = sys.getsizeof(sl)
        size += sys.getsizeof(sl._level)
        node = sl._head
        while node is not sl._tail:
            size += nodesize(node)
            node = node[2]
        size += nodesize(node)
        size += sys.getsizeof(sl._path)  # contains references or None
        size += sys.getsizeof(sl._distance)
        for el in sl._distance:
            size += sys.getsizeof(el)
        return size


class SkipList(object):
    """An indexable skip list.

    A SkipList provides an ordered sequence of key-value pairs. The list is
    always sorted on key and supports O(1) forward iteration. It has O(log N)
    time complexity for key lookup, pair insertion and pair removal anywhere in
    the list. The list also supports O(log N) element access by position.

    The keys of all pairs you add to the skiplist must be be comparable against
    each other, and define the ``<`` and ``<=`` operators.
    """

    UNSET = object()

    p = int((1<<31) / math.e)
    maxlevel = 20

    # Kudos to http://pythonsweetness.tumblr.com/post/45227295342 for some
    # useful tricks, including using a list for the nodes to save memory.

    # Use the built-in Mersenne Twister random number generator. It is more
    # appropriate than SystemRandom because we don't need cryptographically
    # secure random numbers, and we don't want to do a system call to read
    # /dev/urandom for each random number we need (every insertion needs a new
    # random number).

    _rnd = random.Random()
    _rnd.seed(os.urandom(16))

    __slots__ = ('_level', '_head', '_tail', '_path', '_distance', '_last_comparable_node_higher')

#####
# Add _last-comparable_node property in __init__(); needed for floor() and 
# ceiling() (add to __slots__, too)
#####
    def __init__(self):
        self._level = 1
        self._head = self._new_node(self.maxlevel, None, None)
        self._tail = self._new_node(self.maxlevel, None, None)
        for i in range(self.maxlevel):
            self._head[2+i] = self._tail
        self._path = [None] * self.maxlevel
        self._distance = [None] * self.maxlevel

    def _new_node(self, level, key, value):
        # Node layout: [key, value, next*LEVEL, skip?]
        # The "skip" element indicates how many nodes are skipped by the
        # highest level incoming link.
        if level == 1:
            return [key, value, None]
        else:
            return [key, value] + [None]*level + [0]

    def _random_level(self):
        # Exponential distribution as per Pugh's paper.
        l = 1
        maxlevel = min(self.maxlevel, self.level+1)
        while l < maxlevel and self._rnd.getrandbits(31) < self.p:
            l += 1
        return l

    def _create_node(self, key, value):
        # Create a new node, updating the list level if required.
        level = self._random_level()
        if level > self.level:
            self._tail[-1] = len(self)
            self._level = level
            self._path[level-1] = self._head
            self._distance[level-1] = 0
        return self._new_node(level, key, value)

    def _find_lt(self, key):
        # Find path to last node < key
        node = self._head
        distance = 0
        for i in reversed(range(self.level)):
#####
# Keys are partially ordered, so if two are not comparable (because they
# share no sigla), the comparator in S_Key will raise a ComparisonError,
# which we'll trap and then skip to the next key)
#
# We add and use last_comparable_node for our floor() and ceiling(), so
# that we don't return a node that is unordered with respect to the one we
# are querying
#
# Same modification in _find_lt() and _find_lte()
#####
            nnode = node[2+i]
#             while nnode is not self._tail and nnode[0] < key:
#                 nnode, node = nnode[2+i], nnode
#                 distance += 1 if i == 0 else node[-1]
            while nnode is not self._tail:
                try:
                    if nnode[0] < key:
                        nnode, node = nnode[2+i], nnode
                        distance += 1 if i == 0 else node[-1]
                        last_comparable_node = node
                    else:
                        break
                except ComparisonError:
                    nnode = nnode[2+i]
                    continue
            self._path[i] = node
            self._distance[i] = distance
        self._last_comparable_node_higher = nnode


    def _find_lte(self, key):
        # Find path to last node <= key
        node = self._head
        distance = 0
        for i in reversed(range(self.level)):
            nnode = node[2+i]
#             while nnode is not self._tail and nnode[0] <= key:
#                 nnode, node = nnode[2+i], nnode
#                 distance += 1 if i == 0 else node[-1]
            while nnode is not self._tail:
                try:
                    if nnode[0] <= key:
                        nnode, node = nnode[2+i], nnode
                        distance += 1 if i == 0 else node[-1]
                        last_comparable_node = node
                    else:
                        break
                except ComparisonError:
                    nnode = nnode[2+i]
                    continue
            self._path[i] = node
            self._distance[i] = distance
        self._last_comparable_node_higher = nnode

    def _find_pos(self, pos):
        # Create path to node at pos.
        node = self._head
        distance = 0
        for i in reversed(range(self.level)):
            nnode = node[2+i]
            ndistance = distance + (1 if i == 0 else nnode[-1])
            while nnode is not self._tail and ndistance <= pos:
                nnode, node, distance = nnode[2+i], nnode, ndistance
                ndistance += 1 if i == 0 else nnode[-1]
            self._path[i] = node
            self._distance[i] = distance

    def _insert(self, node):
        # Insert a node in the list. The _path and _distance must be set.
        path, distance = self._path, self._distance
        # Update pointers
        level = max(1, len(node) - 3)
        for i in range(level):
            node[2+i] = path[i][2+i]
            path[i][2+i] = node
        if level > 1:
            node[-1] = 1 + distance[0] - distance[level-1]
        # Update skip counts
        node = node[2]
        i = 2; j = min(len(node) - 3, self.level)
        while i <= self.level:
            while j < i:
                node = node[i]
                j = min(len(node) - 3, self.level)
            node[-1] -= distance[0] - distance[j-1] if j <= level else -1
            i = j+1

    def _remove(self, node):
        # Remove a node. The _path and _distance must be set.
        path, distance = self._path, self._distance
        level = max(1, len(node) - 3)
        for i in range(level):
            path[i][2+i] = node[2+i]
        # Update skip counts
        value = node[1]
        node = node[2]
        i = 2; j = min(len(node) - 3, self.level)
        while i <= self.level:
            while j < i:
                node = node[i]
                j = min(len(node) - 3, self.level)
            node[-1] += distance[0] - distance[j-1] if j <= level else -1
            i = j+1
        # Reduce level if last node on current level was removed
        while self.level > 1 and self._head[1+self.level] is self._tail:
            self._level -= 1
            self._tail[-1] += self._tail[-1] - len(self)
        return value

    # PUBLIC API ...

    @property
    def level(self):
        """The current level of the skip list."""
        return self._level

    def insert(self, key, value):
        """Insert a key-value pair in the list.

        The pair is inserted at the correct location so that the list remains
        sorted on *key*. If a pair with the same key is already in the list,
        then the pair is appended after all other pairs with that key.
        """
        self._find_lte(key)
        node = self._create_node(key, value)
        self._insert(node)

    def replace(self, key, value):
        """Replace the value of the first key-value pair with key *key*.

        If the key was not found, the pair is inserted.
        """
        self._find_lt(key)
        node = self._last_comparable_node_higher
        if node is self._tail or key < node[0]:
            node = self._create_node(key, value)
            self._insert(node)
        else:
            node[1] = value

    def clear(self):
        """Remove all key-value pairs."""
        for i in range(self.maxlevel):
            self._head[2+i] = self._tail
            self._tail[-1] = 0
        self._level = 1

    def __len__(self):
        """Return the number of pairs in the list."""
        dist = 0
        idx = self.level + 1
        node = self._head[idx]
        while node is not self._tail:
            dist += node[-1] if idx > 2 else 1
            node = node[idx]
        dist += node[-1]
        return dist

    __bool__ = __nonzero__ = lambda self: len(self) > 0

    def __repr__(self):
        return type(self).__name__ + '((' + repr(list(self.items()))[1:-1] + '))'

    def items(self, start=None, stop=None):
        """Return an iterator yielding pairs.

        If *start* is specified, iteration starts at the first pair with a key
        that is larger than or equal to *start*. If not specified, iteration
        starts at the first pair in the list.

        If *stop* is specified, iteration stops at the last pair that is
        smaller than *stop*. If not specified, iteration end with the last pair
        in the list.
        """
        if start is None:
            node = self._head[2]
        else:
            self._find_lt(start)
            node = self._last_comparable_node_higher
        while node is not self._tail and (stop is None or node[0] < stop):
            yield (node[0], node[1])
            node = node[2]

    __iter__ = items

    def keys(self, start=None, stop=None):
        """Like :meth:`items` but returns only the keys."""
        return (item[0] for item in self.items(start, stop))

    def values(self, start=None, stop=None):
        """Like :meth:`items` but returns only the values."""
        return (item[1] for item in self.items(start, stop))

    def popitem(self):
        """Removes the first key-value pair and return it.

        This method raises a ``KeyError`` if the list is empty.
        """
        node = self._head[2]
        if node is self._tail:
            raise KeyError('list is empty')
        self._find_lt(node[0])
        self._remove(node)
        return (node[0], node[1])

    # BY KEY API ...

    def search(self, key, default=None):
        """Find the first key-value pair with key *key* and return its value.

        If the key was not found, return *default*. If no default was provided,
        return ``None``. This method never raises a ``KeyError``.
        """
        self._find_lt(key)
        node = self._last_comparable_node_higher
        if node is self._tail or key < node[0]:
            return default
        return node[1]

    def remove(self, key):
        """Remove the first key-value pair with key *key*.

        If the key was not found, a ``KeyError`` is raised.
        """
        self._find_lt(key)
        node = self._last_comparable_node_higher
        if node is self._tail or key < node[0]:
            raise KeyError('{!r} is not in list'.format(key))
        self._remove(node)

    def pop(self, key, default=UNSET):
        """Remove the first key-value pair with key *key*.

        If a pair was removed, return its value. Otherwise if *default* was
        provided, return *default*. Otherwise a ``KeyError`` is raised.
        """
        self._find_lt(key)
        node = self._last_comparable_node_higher
        if node is self._tail or key < node[0]:
            if default is self.UNSET:
                raise KeyError('key {!r} not in list')
            return default
        self._remove(node)
        return node[1]

    def __contains__(self, key):
        """Return whether *key* is contained in the list."""
        self._find_lt(key)
        node = self._last_comparable_node_higher
        return node is not self._tail and not key < node[0]

    def index(self, key, default=UNSET):
        """Find the first key-value pair with key *key* and return its position.

        If the key is not found, return *default*. If default was not provided,
        raise a ``KeyError``
        """
        self._find_lt(key)
        node = self._last_comparable_node_higher
        if node is self._tail or key < node[0]:
            if default is self.UNSET:
                raise KeyError('key {!r} not in list'.format(key))
            return default
        return self._distance[0]

    def count(self, key):
        """Return the number of pairs with key *key*."""
        count = 0
        pos = self.index(key, -1)
        if pos == -1:
            return count
        count += 1
        for i in range(pos+1, len(self)):
            if self[i][0] != key:
                break
            count += 1
        return count

    # BY POSITION API ...

    def __getitem__(self, pos):
        """Return a pair by its position.

        If *pos* is a slice, then return a generator that yields pairs as
        specified by the slice.
        """
        size = len(self)
        if isinstance(pos, int):
            if pos < 0:
                pos += size
            if not 0 <= pos < size:
                raise IndexError('list index out of range')
            self._find_pos(pos)
            node = self._path[0][2]
            return (node[0], node[1])
        elif isinstance(pos, slice):
            start, stop = pos.start, pos.stop
            if start is None:
                start = 0
            elif start < 0:
                start += size
            if stop is None:
                stop = size
            elif stop < 0:
                stop += size
            self._find_pos(start)
            def genpairs():
                pos = start; node = self._path[0][2]
                while node is not self._tail and pos < stop:
                    yield (node[0], node[1])
                    node = node[2]; pos += 1
            return genpairs()
        else:
            raise TypeError('expecting int or slice, got {0.__name__!r}'.format(type(pos)))

    def __delitem__(self, pos):
        """Delete a pair by its position."""
        if not isinstance(pos, int):
            raise TypeError('expecting int, got {0.__name__!r}'.format(type(pos)))
        size = len(self)
        if pos < 0:
            pos += size
        if not 0 <= pos < size:
            raise IndexError('list index out of range')
        self._find_pos(pos)
        node = self._path[0][2]
        self._remove(node)

    def __setitem__(self, pos, value):
        """Set a value by its position."""
        if not isinstance(pos, int):
            raise TypeError('expecting int, got {0.__name__!r}'.format(type(pos)))
        size = len(self)
        if pos < 0:
            pos += size
        if not 0 <= pos < size:
            raise IndexError('list index out of range')
        self._find_pos(pos)
        node = self._path[0][2]
        node[1] = value

In [4]:
from dataclasses import dataclass

@dataclass
class SkiplistNode:
    """Declare SkipListNode class for export in dumpNodes()"""
    level: int
    key: str
    value: list
    name: str = None

def getSkiplistNodeData(node):
    level = max(1, len(node) - 3)
    key = node[0]
    value = [node[1]][0]
    return(level, key, value)

def dumpNodes(sl):
    SkiplistNodes = []
    node = sl._head
    nodeValues = getSkiplistNodeData(node)
    SkiplistNodes.append(SkiplistNode(level=nodeValues[0], key=nodeValues[1], value=nodeValues[2], name="head"))
    node = node[2]
    while node is not sl._tail:
        nodeValues = getSkiplistNodeData(node)
        SkiplistNodes.append(SkiplistNode(level=nodeValues[0], key=nodeValues[1], value=nodeValues[2]))
        node = node[2]
    nodeValues = getSkiplistNodeData(node)
    SkiplistNodes.append(SkiplistNode(level=nodeValues[0], key=nodeValues[1], value=nodeValues[2], name="tail"))
    return SkiplistNodes

def floor(sl: SkipList, key: str) -> str:
    """Return highest key value that is less than the new key that we seek to place

    Assumption: keys are unique
    Note: lt, not le (unlike Java)"""
    sl._find_lte(key) # change state (eek!)
    return sl._path[0][0] if sl._path[0] else None

def ceiling(sl: SkipList, key: str) -> str:
    """Return lowest value that is greater than the new key that we seek to place

    Assumption: keys are unique
    Note: gt, not ge (unlike Java)"""

    sl._find_lte(key) # change state (eek!)
    return sl._last_comparable_node_higher[0] if sl._last_comparable_node_higher else None

In [5]:
from functools import total_ordering

# Find all witness that are present on both items (ignore those on only one item)
#     If none, raise an error
#     If some, compare in arbitrary but consistent order
#         If first pair differs, return result
#         If first pair matches, check next pair

class ComparisonError(Exception):
    def __init__(self, message):
        self.message = message
    def __str__(self):
        return self.message

@total_ordering
class VG_node():
    """Variant graph node, used as key in skiplist

    We don't care about the value in the skiplist; the node contains all information"""
    def __init__(self, block_id=None, token_offset=None, **kwargs):
        self.__dict__.update(kwargs)
        self._sigla = {key for key in kwargs.keys()}
        self.block_id = block_id
        self.token_offset = token_offset
    def __repr__(self):
        return "|".join([":".join([str(key), str(getattr(self, key))]) for key in self.sigla()])
    def __setitem__(self, key, value):
        self._sigla.add(key)
        self.__dict__[key] = value
    def __getitem__(self, key):
        return self.__dict__[key]
    def __contains__(self, key):
        return key in self.__dict__
    def sigla(self):
        return self._sigla
    def find_shared_sigla(self, other):
        """Return shared sigla for use in comparison, raise error if none"""
        shared_sigla = sorted(set(self.sigla()).intersection(set(other.sigla())))
        if not shared_sigla:
            raise ComparisonError("No shared sigla; cannot be compared")
        return shared_sigla
    def __eq__(self, other):
        shared_sigla = self.find_shared_sigla(other)
        return [getattr(self, siglum) for siglum in shared_sigla] == [getattr(other, siglum) for siglum in shared_sigla]
    def __lt__(self, other):
        shared_sigla = self.find_shared_sigla(other)
#         print("We are comparing the following sigla: "+str(shared_sigla))
#         print(str(self) + " : " + str(other))
        return [getattr(self, siglum) for siglum in shared_sigla] < [getattr(other, siglum) for siglum in shared_sigla]

In [6]:
# we can use token-string frequency (overall) to align rare tokens first within each block
from collections import Counter
all_token_frequencies = Counter(token_array)
# exclude singletons (not in blocks)
token_frequencies = {k:v for k, v in all_token_frequencies.items() if v > 1}
# pp.pprint(token_frequencies) # take a look

# Skiplist setup code above, now create vg (variant graph) skiplist below

In [7]:
vg = SkipList()
tail_value = len(token_array) + 1
# TODO: Adjust START and END to include all witnesses
# TODO; Track and visualize transpositions
vg.insert(VG_node(w0=-1, w1=-1, w2=-1, w3=-1), ['[START]'])
vg.insert(VG_node(w0=tail_value, w1=tail_value, w2=tail_value, w3=tail_value), ['[END]'])
# To update a VG node, try modifying it in place (i.e., modify the skiplist key in place)
# If that breaks, remove it, update it, and reinsert it (verify remove() code, since we modified insert())
# dumpNodes(vg)

# for debugging purposes we keep track of a (block_id, token_off_set_pos_in_block) -> token map
tuple_to_token_map = {}

for index, prioritized_block in enumerate(prioritized_blocks):
    # add each token from the block individually
    # first block: Block(token_count=2, start_position=4, end_position=6, all_start_positions=[2, 9, 18], witnesses={0, 1, 2}, witness_count=3, frequency=3)
    if index: # limit number of blocks for testing; open the limit for production
        for start_position in prioritized_block.all_start_positions:
            witness_token_positions = range(start_position, start_position + prioritized_block.token_count)
            
            tmp = sorted(witness_token_positions, key = lambda x: token_frequencies[token_array[x]], reverse=True)
            # print(tmp)
            
            # The code below aims to identify transpositions, but proved less helpful than we thought because
            # our alignment was poor. We attempt to improve the alignment by processing tokens within a block
            # not from left to righht, but from least to most frequent in the corpus. We do that in our new tmp
            # variable, above, which we continue to develop in skiplist-04.ipynb. We retain skiplist-03.ipynb
            # so that we can recover the transposition detection trigger and report later.

            # recreate generator temporarily
            witness_token_positions = range(start_position, start_position + prioritized_block.token_count)
            
            current_witness = 'w' + str(token_membership_array[start_position])
            for token_offset_in_block, witness_token_pos in enumerate(witness_token_positions):
                # get all tokens for each witness and retrieve token string and witness identifier
                # current_token = token_array[witness_token_pos] # sanity check
                # print(current_token, current_witness) # sanity check
                # https://stackoverflow.com/questions/42845252/python-how-to-pass-argument-name-using-variable
                key_to_check = VG_node(index, token_offset_in_block,**{current_witness: witness_token_pos})
                # if not tuple_to_token_map contains (index, token_offset_in_block) put it in there
                # If it is there, check whether it is the same token/string.
                string_to_look_for = token_array[witness_token_pos]

                if (index, token_offset_in_block) in tuple_to_token_map:
                    # check whether it is the same token
                    if (tuple_to_token_map[(index, token_offset_in_block)]!=string_to_look_for):
                        raise Exception("Token mismatch on a node!")
                else:
                    tuple_to_token_map[(index, token_offset_in_block)] = string_to_look_for




                # print('we are checking ' + str(key_to_check))
                if key_to_check not in vg:
                    # print('we need to add  ', str(key_to_check))
                    floor_node = floor(vg, key_to_check)
                    # print('floor =         ', floor_node)
                    ceiling_node = ceiling(vg, key_to_check)
                    # print('ceiling =       ', ceiling_node)
                    # create a list of the items between the floor and ceiling
                    # floor is inclusive, ceiling is exclusive in the skiplist items method. 
                    gen = vg.items(floor_node, ceiling_node)
                    # we have to skip the floor node
                    zzz = next(gen)
                    current_slice = list(gen)
                    # print(current_slice)
                    if not len(current_slice): # floor and ceiling are adjacent, so insert between them
                        # insertion position is automatic for a skiplist (if the comparator is doing its job)
                        vg.insert(key_to_check, [witness_token_pos])
                    else: # do stuff
                        # print()
                        # #####
                        # Are any of the slice nodes candidates for placing new token?
                        # Blocks contain information about all occurrences of pattern.
                        # If token is part of the same block, it can be added to it?
                        #
                        # We already did string matching when we constructed blocks. Now
                        # find intersection of:
                        #   1) nodes between floor and ceiling and
                        #   2) instances of block
                        # If there is an intersection, that's the node with which to align.
                        # To do that:
                        #   1) Remove the existing node
                        #   2) Modify its key (and value?) and insert it
                        # We think it will go back into the same place, but if not, the
                        # skiplist should place it correctly (topologically) anyway.
                        # NB: We do not modify the key or value in place!
                        #
                        # Possibilities:
                        #   1) none
                        #      Insert new node
                        #   2) exactly one
                        #      Add to it
                        #   3) more than one (if there is repetition)
                        #      Defer consideration until we encounter one
                        #
                        # print(f"{slice_length=}") # diagnostic
                        #
                        # Is it situation 1, 2, or 3?
                        #
                        # token_to_block_dict is a multi-valued dictionary:
                        #   keys are token offsets
                        #   values are lists of block offsets in  prioritized_blocks list, which serve as block ids
                        for current_node in current_slice: # look at all vg nodes between floor and ceiling; tuple of vg_node, [offsets]
                            # print("We are visiting "+str(current_node)+" as a potential candidate for alignment")
                            # index is for block we're trying to place; is in graph between floor and ceiling?
                            blocks_for_vg_node = [] # blocks associated with current variant graph node
                            for current_token_offset in current_node[1]: # for each node, look at each of its values
                                blocks_for_current_token = token_to_block_dict[current_token_offset] # non-empty list of integers
                                blocks_for_vg_node.extend(blocks_for_current_token) # list may include duplicates; do we care?
                            # print(f"{blocks_for_vg_node=}")
                            #     if token_offset_in_block == current_node[0].token_offset and index in blocks_for_vg_node: # does the current block, inside the slice, contain the node we care about?
                            if token_array[current_node[1][0]] == token_array[witness_token_pos]:
                                new_key_siglum = 'w' + str(token_membership_array[witness_token_pos]) # witness siglum for new token instance
                                new_key = current_node[0] # start building new key from old key
                                if new_key_siglum in new_key:
                                    raise Exception("The key is already on this node")
                                new_key[new_key_siglum] = witness_token_pos # add new token to key
                                new_value = current_node[1].copy()
                                new_value.append(witness_token_pos) # add new token to value, too
                                # print(new_key, new_value)

                                # Debug: CHECK whether the token content is the same when adding an offset to an existing node.
                                string_already_on_the_node = token_array[current_node[1][0]]
                                string_to_be_placed = token_array[witness_token_pos]
                                if new_key_siglum == "w0" and string_already_on_the_node != string_to_be_placed:
                                    raise Exception(string_already_on_the_node+" != "+string_to_be_placed+" Nodes to check: "+str(current_slice)+" floor: "+str(floor_node)+" : "+str(ceiling_node))

                                
                                
                                vg.remove(current_node[0]) # found one; remove matching node by key
                                vg.insert(new_key, new_value) # update key and value, insert
                                break
                        else:
                            vg.insert(key_to_check, [witness_token_pos]) # didn't find one, insert new

# place tokens that are not parts of blocks
unplaced_tokens = [offset for offset in range(len(token_array))
                   if offset not in token_to_block_dict
                   and isinstance(token_membership_array[offset], int)]
for unplaced_token in unplaced_tokens:
    new_node = VG_node(**{'w' + str(
token_membership_array[unplaced_token]): unplaced_token})
    vg.insert(new_node, [unplaced_token])

# dumpNodes(vg) # take a look
# pp.pprint(prioritized_blocks) # take a look at this, too

[240, 246, 254, 274, 283, 292, 338, 381, 400, 407, 238, 241, 255, 272, 289, 293, 309, 351, 366, 397, 237, 320, 329, 349, 396, 402, 251, 295, 301, 313, 344, 409, 262, 322, 372, 387, 258, 297, 302, 308, 336, 350, 356, 235, 244, 247, 278, 286, 298, 299, 303, 317, 330, 334, 353, 354, 376, 391, 326, 359, 239, 243, 249, 250, 253, 264, 273, 275, 276, 279, 280, 281, 284, 288, 294, 296, 305, 311, 315, 318, 323, 325, 337, 339, 346, 355, 357, 361, 365, 373, 374, 392, 393, 398, 399, 401, 404, 261, 236, 242, 245, 248, 252, 256, 257, 259, 260, 263, 265, 266, 267, 268, 270, 271, 277, 282, 285, 287, 290, 291, 300, 304, 306, 307, 310, 312, 314, 316, 319, 321, 327, 328, 331, 332, 333, 335, 340, 341, 342, 343, 345, 347, 348, 352, 358, 360, 362, 363, 364, 367, 368, 369, 370, 371, 375, 377, 378, 379, 380, 382, 383, 384, 385, 386, 388, 389, 390, 394, 395, 403, 405, 406, 408, 269, 324]
[473, 479, 487, 507, 516, 525, 571, 614, 633, 640, 471, 474, 488, 505, 522, 526, 542, 584, 599, 630, 470, 553, 562, 582, 629

[240, 246, 254, 274, 283, 292, 338, 381, 400, 407, 238, 241, 255, 272, 289, 293, 309, 351, 366, 397, 237, 320, 329, 349, 396, 402, 251, 295, 301, 313, 344, 409, 262, 322, 372, 387, 258, 297, 302, 308, 336, 350, 356, 244, 247, 278, 286, 298, 299, 303, 317, 330, 334, 353, 354, 376, 391, 326, 359, 239, 243, 249, 250, 253, 264, 273, 275, 276, 279, 280, 281, 284, 288, 294, 296, 305, 311, 315, 318, 323, 325, 337, 339, 346, 355, 357, 361, 365, 373, 374, 392, 393, 398, 399, 401, 404, 261, 236, 242, 245, 248, 252, 256, 257, 259, 260, 263, 265, 266, 267, 268, 270, 271, 277, 282, 285, 287, 290, 291, 300, 304, 306, 307, 310, 312, 314, 316, 319, 321, 327, 328, 331, 332, 333, 335, 340, 341, 342, 343, 345, 347, 348, 352, 358, 360, 362, 363, 364, 367, 368, 369, 370, 371, 375, 377, 378, 379, 380, 382, 383, 384, 385, 386, 388, 389, 390, 394, 395, 403, 405, 406, 408, 269, 324]
[473, 479, 487, 507, 516, 525, 571, 614, 633, 640, 471, 474, 488, 505, 522, 526, 542, 584, 599, 630, 470, 553, 562, 582, 629, 635

[507, 516, 525, 571, 614, 633, 640, 488, 505, 522, 526, 542, 584, 599, 630, 553, 562, 582, 629, 635, 528, 534, 546, 577, 642, 495, 555, 605, 620, 491, 530, 535, 541, 569, 583, 589, 511, 519, 531, 532, 536, 550, 563, 567, 586, 587, 609, 624, 559, 592, 497, 506, 508, 509, 512, 513, 514, 517, 521, 527, 529, 538, 544, 548, 551, 556, 558, 570, 572, 579, 588, 590, 594, 598, 606, 607, 625, 626, 631, 632, 634, 637, 494, 489, 490, 492, 493, 496, 498, 499, 500, 501, 503, 504, 510, 515, 518, 520, 523, 524, 533, 537, 539, 540, 543, 545, 547, 549, 552, 554, 560, 561, 564, 565, 566, 568, 573, 574, 575, 576, 578, 580, 581, 585, 591, 593, 595, 596, 597, 600, 601, 602, 603, 604, 608, 610, 611, 612, 613, 615, 616, 617, 618, 619, 621, 622, 623, 627, 628, 636, 638, 639, 641, 502, 557]
[743, 752, 761, 807, 850, 869, 876, 724, 741, 758, 762, 778, 820, 835, 866, 789, 798, 818, 865, 871, 764, 770, 782, 813, 878, 731, 791, 841, 856, 727, 766, 771, 777, 805, 819, 825, 747, 755, 767, 768, 772, 786, 799, 803, 822

In [8]:
# print(" ".join(token_array[704:738])) # w3
# print(" ".join(token_array[235:269])) # w1
# print(" ".join(token_array[468:502])) # w2
# print(" ".join(token_array[1:35])) # w0 (constructed; does not match the others)

# Import code to generate SVG of skiplist from render_skiplist_as_svg.ipynb

In [9]:
%run render_skiplist_as_svg.ipynb

# Import code to generate SVG of variant graph from render_variant_graph_as_svg.ipynb

Rendering instruction in imported module doesn’t work; we need it issue it here

In [10]:
%run render_variant_graph_as_svg.ipynb
SVG(a.view())

In [11]:
# for index,node in enumerate(dumpNodes(vg)):
#     pp.pprint((index,node))

In [12]:
prioritized_blocks[:10]

[Block(token_count=176, start_position=37, end_position=39, all_start_positions=[234, 467, 703], witnesses={1, 2, 3}, witness_count=3, frequency=3),
 Block(token_count=175, start_position=895, end_position=897, all_start_positions=[235, 468, 704], witnesses={1, 2, 3}, witness_count=3, frequency=3),
 Block(token_count=174, start_position=401, end_position=403, all_start_positions=[236, 469, 705], witnesses={1, 2, 3}, witness_count=3, frequency=3),
 Block(token_count=173, start_position=791, end_position=793, all_start_positions=[237, 470, 706], witnesses={1, 2, 3}, witness_count=3, frequency=3),
 Block(token_count=172, start_position=715, end_position=717, all_start_positions=[238, 471, 707], witnesses={1, 2, 3}, witness_count=3, frequency=3),
 Block(token_count=171, start_position=369, end_position=371, all_start_positions=[239, 472, 708], witnesses={1, 2, 3}, witness_count=3, frequency=3),
 Block(token_count=170, start_position=491, end_position=493, all_start_positions=[240, 473, 709

In [13]:
# for index,block in enumerate(prioritized_blocks):
#     first_token = block.all_start_positions[0]
#     first_five_tokens = token_array[first_token: first_token + 4] # wrong with token_count < 5
#     witnesses = block.witnesses
#     token_count = block.token_count
#     if 'when' in first_five_tokens:
#         pp.pprint((index, first_five_tokens, witnesses, token_count))

(50, ['when', 'the', 'organisation', 'has'], {0, 1, 2, 3}, 54)
(311, ['variation;', 'and', 'that', 'when'], {0, 1, 2}, 57)
(320, ['and', 'that', 'when', 'the'], {0, 1, 2}, 56)
(327, ['that', 'when', 'the', 'organisation'], {0, 1, 2}, 55)
(328, ['when', 'the', 'organisation', 'has'], {0, 2, 3}, 55)
(578, ['variation;', 'and', 'that', 'when'], {0, 2}, 58)
(580, ['and', 'that', 'when', 'the'], {0, 2}, 57)
(585, ['that', 'when', 'the', 'organisation'], {0, 2}, 56)
(624, ['variation;', 'and', 'that', 'when'], {0, 1, 2, 3}, 2)


In [14]:
# for index,value in enumerate(token_array):
#     if value == 'when':
#         print(token_membership_array[index], token_array[index - 2 : index + 3])

0 ['and', 'that', 'when', 'the', 'organisation']
1 ['and', 'that', 'when', 'the', 'organisation']
2 ['and', 'that', 'when', 'the', 'organisation']
3 ['and', 'that,', 'when', 'the', 'organisation']


# Start here

1. Add block identifier to VG token when token is created:  store 1) block id, 2) offset within block. This won't track all blocks associated with the token, but it will record the block we were processing when the token was first created.
1. Create hyperedges that group *sibling* (*transposition*) VG nodes as they're created. Decide what to do about them later. They'll tell us which nodes are related, but not necessarily which specifically are transposed with which others.

# The offset within a block is calculated in exact positions in line 17
We would need to record the relative offset next to the block identifier, so the exact position - the start position.

# Priority

1. We already prioritize blocks
1. We can also prioritize tokens with blocks 

In [15]:
# import itertools
# groups = []
# unique_keys = []
# result = itertools.groupby(vg.keys(), key=lambda x: (x.block_id, x.token_offset))
# # pp.pprint(list(result))
# for k, g in result:
#     if k != (None, None):
#             groups.append(list(g))
#             unique_keys.append(k)
# pp.pprint(list(zip(unique_keys, groups)))

# for vg_node in vg:
#     if vg_node.block_id:
#         print(vg_node.block_id, vg_node.token_offset, vg_node)

# groups = []
# uniquekeys = []
# data = sorted(data, key=keyfunc)
# for k, g in groupby(data, keyfunc):
#     groups.append(list(g))      # Store group iterator as a list
#     uniquekeys.append(k)

In [16]:
# Sort vg by block_id and first token offset
# Return block_id, first token offset, and token strings with numbers
# vg_nodes = sorted([(k.block_id, k.token_offset, k) for k in vg.keys() if k.block_id != None], key=lambda x: (x[0], x[1]))
# for vg_node in vg_nodes:
#     key = vg_node[2]
#     print((vg_node[0:2], [token_array[key[i]] for i in ['w0', 'w1', 'w2', 'w3'] if i in key]))

((1, 0), ['we', 'we', 'we', 'we'])
((1, 1), ['look', 'look', 'look', 'look'])
((1, 2), ['to', 'to', 'to', 'to'])
((1, 3), ['the', 'the', 'the', 'the'])
((1, 4), ['individuals', 'individuals', 'individuals', 'individuals'])
((1, 5), ['of', 'of', 'of', 'of'])
((1, 6), ['the', 'the', 'the', 'the'])
((1, 7), ['same', 'same', 'same', 'same'])
((1, 8), ['variety', 'variety', 'variety', 'variety'])
((1, 9), ['or', 'or', 'or', 'or'])
((1, 10), ['sub-variety', 'sub-variety', 'sub-variety', 'sub-variety'])
((1, 11), ['of', 'of', 'of', 'of'])
((1, 12), ['our', 'our', 'our', 'our'])
((1, 13), ['older', 'older', 'older', 'older'])
((1, 14), ['cultivated', 'cultivated', 'cultivated', 'cultivated'])
((1, 15), ['plants', 'plants', 'plants', 'plants'])
((1, 16), ['and', 'and', 'and', 'and'])
((1, 17), ['animals,', 'animals,', 'animals,', 'animals,'])
((1, 18), ['one', 'one', 'one', 'one'])
((1, 19), ['of', 'of', 'of', 'of'])
((1, 20), ['the', 'the', 'the', 'the'])
((1, 21), ['first', 'first', 'first', 


((50, 25), ['ceasing'])
((50, 25), ['ceasing', 'ceasing', 'ceasing'])
((50, 26), ['to'])
((50, 26), ['to', 'to', 'to'])
((50, 27), ['be'])
((50, 27), ['be', 'be', 'be'])
((50, 28), ['variable'])
((50, 28), ['variable', 'variable', 'variable'])
((50, 29), ['under'])
((50, 29), ['under', 'under', 'under'])
((50, 30), ['cultivation.'])
((50, 30), ['cultivation.', 'cultivation.', 'cultivation.'])
((50, 31), ['Our'])
((50, 31), ['Our', 'Our', 'Our'])
((50, 32), ['oldest'])
((50, 32), ['oldest', 'oldest', 'oldest'])
((50, 33), ['cultivated'])
((50, 33), ['cultivated', 'cultivated', 'cultivated'])
((50, 34), ['plants,'])
((50, 34), ['plants,', 'plants,', 'plants,'])
((50, 35), ['such'])
((50, 35), ['such', 'such', 'such'])
((50, 36), ['as'])
((50, 36), ['as', 'as', 'as'])
((50, 37), ['wheat,'])
((50, 37), ['wheat,', 'wheat,', 'wheat,'])
((50, 38), ['still'])
((50, 38), ['still', 'still', 'still'])
((50, 39), ['often'])
((50, 39), ['often', 'often', 'often'])
((50, 40), ['yield'])
((50, 40), 

In [17]:
# Low-priority blocks are more likely to be transposed
# Blocks appear to be okay; check how we add them to the variant graph
# for pb in prioritized_blocks:
#     print(set([token_array[sp] for sp in pb.all_start_positions]))

In [18]:
# pp.pprint(list(enumerate(token_array)))

In [19]:
# tuple_to_token_map